In [ ]:
import scanpy as sc
import os
# Path to your file
data_dir = "./data"
fname = os.path.join(data_dir, "HS27_fibroblast_CRISPRa_mean_pop.h5ad")

print("Loading:", fname)
adata = sc.read_h5ad(fname)

print("\n===== Basic Info =====")
print(adata)

print("\n===== Dimensions =====")
print(f"Number of (obs):        {adata.n_obs}")
print(f"Number of genes (var):      {adata.n_vars}\n")

print("X shape:", adata.X.shape)

print("\n===== obs head =====")
print(adata.obs.head())

print("\n===== var head =====")
print(adata.var.head())


Loading: ./data/HS27_fibroblast_CRISPRa_mean_pop.h5ad

===== Basic Info =====
AnnData object with n_obs × n_vars = 10916 × 4914
    obs: 'protospacer', 'target_gene', 'target_expr', 'active', 'cluster', 'cluster_name', 'cluster_description', 'gene_driven', 'sequence_driven', 'sequence_suffix', 'bulk_expr', 'expressed', 'p_expressed', 'de_genes', 'strength', 'cell_count', 'target_gene_id', 'masked_active', 'masked_cluster', 'masked_cluster_name', 'masked_cluster_description', 'masked_cluster_size', 'masked_cluster_num_genes', 'stricter_masked_cluster', 'stricter_masked_cluster_description'
    var: 'feature_types', 'genome', 'mean', 'in_matrix', 'std', 'cv', 'fano', 'pairwise_p_cost', 'mean_adjusted_pairwise_p_cost', 'excess_cv', 'pairwise_chosen', 'gene_id'
    layers: 'adj_p', 'masked', 'p'

===== Dimensions =====
Number of (obs):        10916
Number of genes (var):      4914

X shape: (10916, 4914)

===== obs head =====
                                    protospacer target_gene  tar

In [ ]:
unique_target_genes = adata.obs['target_gene'].unique()
print(f"Unique target genes ({len(unique_target_genes)}):")
print(unique_target_genes)

Unique target genes (1837):
['AATF', 'ABL1', 'ACTL6A', 'ADAR', 'ADNP2', ..., 'ZXDA', 'ZXDB', 'ZXDC', 'ZZZ3', 'off-target']
Length: 1837
Categories (1837, object): ['AATF', 'ABL1', 'ACTL6A', 'ADAR', ..., 'ZXDB', 'ZXDC', 'ZZZ3', 'off-target']


In [ ]:
import pandas as pd
from pyfaidx import Fasta
import gzip
from tqdm import tqdm

def extract_and_save_sequences(
    h5ad_path, 
    gtf_path, 
    fasta_path, 
    output_path="gene_sequences.csv", 
    window_size=6000
):
    """
    Step 2 complete: extract DNA sequences centered at TSS and save them.
    
    Args:
        h5ad_path: Your .h5ad data file path (used to get the 4914 gene IDs to process)
        gtf_path: Path to the downloaded Homo_sapiens.GRCh38.110.gtf.gz
        fasta_path: Path to the downloaded Homo_sapiens.GRCh38.dna.primary_assembly.fa
        output_path: Path to save results (.csv)
        window_size: Extraction length (default 6000, i.e., TSS ± 3000bp)
    """
    
    print(f"1. Loading gene list from {h5ad_path}...")
    adata = sc.read_h5ad(h5ad_path)
    # Get all gene_id values from the var table in h5ad
    target_gene_ids = set(adata.var['gene_id'].tolist())
    print(f"   Total genes to find: {len(target_gene_ids)}")

    print(f"2. Parsing GTF file to find TSS coordinates...")
    # Build a mapping: gene_id -> (chrom, tss, strand)
    gene_coords = {}
    
    # Read GTF line by line; more memory efficient than loading whole thing into Pandas
    open_func = gzip.open if gtf_path.endswith('.gz') else open
    
    with open_func(gtf_path, 'rt') as f:
        for line in tqdm(f, desc="Scanning GTF"):
            if line.startswith('#'): continue
            parts = line.strip().split('\t')
            
            # We only want rows where feature type is 'gene'
            feature_type = parts[2]
            if feature_type != 'gene': continue
            
            # Parse attributes to get gene_id
            attributes = parts[8]
            # Simple string search to extract gene_id "ENSG..."
            if 'gene_id "' not in attributes: continue
            
            start_idx = attributes.find('gene_id "') + 9
            end_idx = attributes.find('"', start_idx)
            gene_id = attributes[start_idx:end_idx]
            
            # Only process genes that are present in our h5ad target list
            if gene_id in target_gene_ids:
                chrom = parts[0]
                start = int(parts[3])
                end = int(parts[4])
                strand = parts[6]
                
                # Define TSS (Transcription Start Site)
                # Plus strand (+): TSS is start; minus strand (-): TSS is end
                tss = start if strand == '+' else end
                
                gene_coords[gene_id] = {
                    'chrom': chrom,
                    'tss': tss,
                    'strand': strand
                }

    found_count = len(gene_coords)
    missing_count = len(target_gene_ids) - found_count
    print(f"   Found coordinates for {found_count} genes. Missing {missing_count} genes.")
    
    print(f"3. Extracting sequences from FASTA...")
    genome = Fasta(fasta_path)
    
    results = []
    half_window = window_size // 2
    
    for gene_id in tqdm(list(target_gene_ids), desc="Extracting"):
        if gene_id not in gene_coords:
            continue
            
        info = gene_coords[gene_id]
        chrom = info['chrom']
        tss = info['tss']
        strand = info['strand']

        if chrom not in genome:
            if f"chr{chrom}" in genome: chrom = f"chr{chrom}"
            else: continue 

        start_extract = max(0, tss - half_window)
        end_extract = tss + half_window
        
        try:
            # Extract sequence
            seq_obj = genome[chrom][start_extract:end_extract]
            
            if strand == '-':
                # For minus strand, get the reverse complement
                # For pyfaidx, use minus sign before the object
                sequence = (-seq_obj).seq
            else:
                sequence = seq_obj.seq
                
            if len(sequence) != window_size:
                sequence = sequence.ljust(window_size, 'N')

            results.append({
                'gene_id': gene_id,
                'chrom': chrom,
                'tss': tss,
                'strand': strand,
                'sequence': sequence.upper() # convert to uppercase
            })
            
        except Exception as e:
            print(f"Error extracting {gene_id}: {e}")

    print(f"4. Saving to {output_path}...")
    df = pd.DataFrame(results)
    
    df = df.set_index('gene_id').reindex(adata.var['gene_id'].tolist()).reset_index()
    
    df.to_csv(output_path, index=False)
    print("Done! Example sequence:")
    print(df[['gene_id', 'strand', 'sequence']].head(1))
    return df



In [11]:
H5AD_FILE = "data/Hs27_fibroblast_CRISPRa_mean_pop.h5ad"
GTF_FILE = "data/Homo_sapiens.GRCh38.110.gtf"        
FASTA_FILE = "data/Homo_sapiens.GRCh38.dna.primary_assembly.fa"

df = extract_and_save_sequences(
    h5ad_path=H5AD_FILE,
    gtf_path=GTF_FILE,
    fasta_path=FASTA_FILE,
    output_path="gene_sequences_12kb.csv",
    window_size=12000 # 6kb context
)

1. Loading gene list from data/Hs27_fibroblast_CRISPRa_mean_pop.h5ad...
   Total genes to find: 4914
2. Parsing GTF file to find TSS coordinates...


Scanning GTF: 3421627it [00:04, 769926.26it/s]


   Found coordinates for 4912 genes. Missing 2 genes.
3. Extracting sequences from FASTA...


Extracting: 100%|██████████| 4914/4914 [00:01<00:00, 4909.65it/s]


4. Saving to gene_sequences_12kb.csv...
Done! Example sequence:
           gene_id strand                                           sequence
0  ENSG00000000419      -  GCCTCCCTCTTCCCATTCATTGTGCTCTGGCCTTAGCTGGCTTCTA...


In [12]:
import pandas as pd

# Read the CSV file
df_seq_6kb = pd.read_csv("gene_sequences_6kb.csv")
df_seq_12kb = pd.read_csv("gene_sequences_12kb.csv")
# Check and print the number of genes
num_genes = df_seq_6kb.shape[0]
print(f"Number of genes in gene_sequences_6kb.csv: {num_genes}")


Number of genes in gene_sequences_6kb.csv: 4914


In [ ]:


missing_seq = df_seq_6kb[df_seq_6kb['sequence'].isnull() | (df_seq_6kb['sequence'] == '')]
if not missing_seq.empty:
    print("Genes without sequences:")
    print(missing_seq[['gene_id']])



Genes without sequences:
              gene_id
1334  ENSG00000112096
2080  ENSG00000130723


In [14]:
# Check that all sequences in df_seq_12kb are of length 12,000
seq_lengths = df_seq_12kb['sequence'].str.len()
expected_length = 12000
num_full_length = (seq_lengths == expected_length).sum()
num_total = len(seq_lengths)
print(f"{num_full_length}/{num_total} sequences are exactly {expected_length} bp long.")

# Show summary of sequence lengths
print("Sequence length value counts:")
print(seq_lengths.value_counts().sort_index())

# Optionally: show problematic gene_ids
if (seq_lengths != expected_length).any():
    print("Genes with length different from 12kb:")
    print(df_seq_12kb.loc[seq_lengths != expected_length, ['gene_id', 'sequence']])


4912/4914 sequences are exactly 12000 bp long.
Sequence length value counts:
sequence
12000.0    4912
Name: count, dtype: int64
Genes with length different from 12kb:
              gene_id sequence
1334  ENSG00000112096      NaN
2080  ENSG00000130723      NaN


In [21]:
unique_target_genes[:20]

['AATF', 'ABL1', 'ACTL6A', 'ADAR', 'ADNP2', ..., 'ALX1', 'ALX3', 'ALX4', 'ALYREF', 'ANHX']
Length: 20
Categories (1837, object): ['AATF', 'ABL1', 'ACTL6A', 'ADAR', ..., 'ZXDB', 'ZXDC', 'ZZZ3', 'off-target']